## Ejercicio 3.2
Lectura de datos, trabajando con datos estructurados

Antes de comenzar, es necesario nefinir ¿Qué es un *dataframe*?, es una tabla de datos distribuida en la RAM de los nodos, formada por filas y columnas. 

### Lectura de DataFrames (DF)

Spark necesita un conector espeficio que indique cómo obtener los datos de las fuentes, existe una aplia variedad:
- HDFS
- AWS S3
- JDBC y ODBC
- Kafka

Los formatos disponibles pueden variar, como  archivos JSON, CSV o Parquet.

**¿Cómo leer información en spark?**

Se usa el atributo SparkSession y [spark.read](http://spark.read)  en el que se especifican diferentes opciones. 

- Formato: que puede ser alguna de los que ya se mencionó, CSV, JSON, Parquet etc.
- Esquema:
    - En algunos casos no es necesario
    - Spark puede inferir el esquema con inferSchema=true, conlleva mayor tiempo de lectura
    - Sí no todos los datos serán strings
    - Se puede indicar el schema de forma explícita
- Modo de lectura:
    - permissive: Null para aquellos registros que consideren corruptos de cada fila
    - dropMalformed: Descarta las filas que contienen registros con un formato incorrecto.
    - failFast: Lanza un error cuando encuentra un registro con un formato incorrecto.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructType, \
                              StringType,IntegerType

In [2]:
spark = SparkSession.builder.appName("read-files").getOrCreate()

#### Cargar Archivos de texto

```python

myDF = spark.read.format(<formato>)
 .load("/path/to/hdfs/file") # spark es el objeto SparkSession
 # <formato> puede ser “parquet” | “json” | “csv” | “orc” | “avro”

 ```

In [3]:
rawCoffe = spark.read.format("text").load("./data/raw-coffee.txt")

In [4]:
rawCoffe.show()

+--------------+
|         value|
+--------------+
|   folgers, 10|
|     yuban, 10|
| nespresso, 10|
|     ritual, 4|
|four barrel, 5|
+--------------+



Se observa que no posee la estructura correcta para ser manipulado el Dataframe. Al momento de hacer una lectura de alguna fuente se puede especificar el *Schema* 

**¿Qué es un Schema en Spark?**
> Es la definición de los nombres y tipos de datos que va a tomar un dataframe, es un objeto del tipo StructType compuesto por StructFileds. 

In [5]:
df_coffee_schema =  StructType(
    [
        StructField("name", StringType(),True),
        StructField("value", IntegerType(),True),
    ]
)

In [6]:
df_coffee = spark.read.format("csv").schema(df_coffee_schema) \
            .option("header",True)\
            .option("delimiter",", ")\
            .load("./data/raw-coffee.txt")

In [7]:
df_coffee.show()

+-----------+-----+
|       name|value|
+-----------+-----+
|      yuban|   10|
|  nespresso|   10|
|     ritual|    4|
|four barrel|    5|
+-----------+-----+



In [8]:
df_coffee.printSchema()

root
 |-- name: string (nullable = true)
 |-- value: integer (nullable = true)



### Schema inference 
Spark tiene la capacidad de inferir el tipo de dato de las columnas, sin embargo *no es recomendable*, Spark tiene que leer todo el contenido del archivo, parsear y probar los tipos de datos.

In [12]:
df_coffee_without_schema = spark.read.format("csv").option("inferSchema",True).load("./data/raw-coffee.txt")

In [13]:
df_coffee_without_schema.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: double (nullable = true)



In [14]:
df_coffee_without_schema.show()

+-----------+----+
|        _c0| _c1|
+-----------+----+
|    folgers|10.0|
|      yuban|10.0|
|  nespresso|10.0|
|     ritual| 4.0|
|four barrel| 5.0|
+-----------+----+



A simple vista podemos ver que no ha cargado el nombre de las columnas y el tipo de dato de _c1 es doble (double) en lugar de entero. 

#### Steal the Schema 
El patrón Steal the Schema muestra como extraer el schema inferido de un dataframe almecenarlo en un json y posterior reconstruirlo mediante el método `fromJson()` de StructType

In [15]:
df_coffee_steal_sh = df_coffee.schema

In [20]:
df_coffee_steal_sh.json()

'{"fields":[{"metadata":{},"name":"name","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"integer"}],"type":"struct"}'

In [28]:
import json # importamos Json para hacer el load 

In [26]:
schema_saved = StructType.fromJson( json.loads(df_coffee_steal_sh.json() )) 

In [27]:
schema_saved

StructType([StructField('name', StringType(), True), StructField('value', IntegerType(), True)])